In [1]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_300000_samples_6d.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)
num_classes

5

In [3]:
from src.model_updated import PointNet2Classification 
from torch.optim.lr_scheduler import StepLR
# backbone_params = {"sa_mlps": [[2, 2, 2], [2, 2, 2]], "mlp":[64, 128, 256], "downsample_points": [256, 64], "radii":[0.1, 0.15], "ks":[16, 32], "add_xyz":True}

model = PointNet2Classification(num_classes, backbone_params=None)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5) 

In [4]:
from src.model_utils import count_parameters

print(f"Number of parameters in the model: {count_parameters(model)}")

Number of parameters in the model: 43749


In [ ]:
from src.train_classificator import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


: 

In [ ]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=100, directory="cls_updated_vsmall_new_data_4")

Training: 100%|██████████| 1875/1875 [02:51<00:00, 10.93it/s]


Epoch 1/100 - Total Loss: 0.0611 | Accuracy: 98.63%


Validating: 100%|██████████| 469/469 [00:34<00:00, 13.43it/s]


Validation - Loss: 0.0004 | Accuracy: 99.99%


Training: 100%|██████████| 1875/1875 [02:51<00:00, 10.92it/s]


...

Training: 100%|██████████| 1875/1875 [02:52<00:00, 10.84it/s]


Epoch 64/100 - Total Loss: 0.0000 | Accuracy: 100.00%


Validating: 100%|██████████| 469/469 [00:34<00:00, 13.73it/s]


Validation - Loss: 0.0000 | Accuracy: 100.00%


Training:  31%|███       | 579/1875 [00:51<01:52, 11.52it/s]

In [ ]:
checkpoint = torch.load('cls_updated_vsmall_new_data_1/checkpoint_epoch_10.pth', map_location='cpu')  # Load on CPU to avoid GPU memory issues
saved_state_dict = checkpoint['model_state']

# Print layer names and their shapes
for name, param in saved_state_dict.items():
    print(f"{name}: {param.shape}")

backbone.sa1.conv.0.0.weight: torch.Size([16, 3, 1, 1])
backbone.sa1.conv.0.1.weight: torch.Size([16])
backbone.sa1.conv.0.1.bias: torch.Size([16])
backbone.sa1.conv.0.1.running_mean: torch.Size([16])
backbone.sa1.conv.0.1.running_var: torch.Size([16])
backbone.sa1.conv.0.1.num_batches_tracked: torch.Size([])
backbone.sa1.conv.1.0.weight: torch.Size([16, 16, 1, 1])
backbone.sa1.conv.1.1.weight: torch.Size([16])
backbone.sa1.conv.1.1.bias: torch.Size([16])
backbone.sa1.conv.1.1.running_mean: torch.Size([16])
backbone.sa1.conv.1.1.running_var: torch.Size([16])
backbone.sa1.conv.1.1.num_batches_tracked: torch.Size([])
backbone.sa1.conv.2.weight: torch.Size([16, 16, 1, 1])
backbone.sa1.last_norm.weight: torch.Size([16])
backbone.sa1.last_norm.bias: torch.Size([16])
backbone.sa1.last_norm.running_mean: torch.Size([16])
backbone.sa1.last_norm.running_var: torch.Size([16])
backbone.sa1.last_norm.num_batches_tracked: torch.Size([])
backbone.sa2.conv.0.0.weight: torch.Size([16, 19, 1, 1])
backb